In [0]:
!pip install optuna

In [0]:
import optuna
import torch

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [0]:
class LogisticNet(torch.nn.Module):
    def __init__(self, D_in, D_out, trial):
        super(LogisticNet, self).__init__()
        H = trial.suggest_categorical('hidden_dim', [16, 32, 64, 128])
        dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 1.0)
        self.fc1 = nn.Linear(D_in, H)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(H, D_out)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        h = self.dropout(h)
        out = self.fc2(h)
        return out

In [0]:
def train(model, loss_func, optimizer, trX, trY):
    x = torch.Tensor(trX)
    y = torch.LongTensor(trY)
    optimizer.zero_grad()
    y_pred = model(x)
    loss = loss_func(y_pred, y)
    loss.backward()
    optimizer.step()
    return loss.item()
  
def valid(model, loss_func, valX, valY):
    x = torch.Tensor(valX)
    y = torch.LongTensor(valY)

    outputs = model(x)
    val_loss = loss_func(outputs, y)
    _, predY = torch.max(outputs.data, 1)
    correct = (predY == y.data).sum()
    val_acc = float(correct) / y.size(0)
    return val_loss.item(), val_acc

In [0]:
def create_optimizer(trial):
    optimizer = trial.suggest_categorical('optimizer', ['sgd', 'adam'])
    if optimizer[0] == 'sgd':
      return optim.SGD(model.parameters(), lr=1e-2)
    else:
      return optim.Adam(model.parameters())

def create_optimizer(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    return optim.Adam(model.parameters(), lr=learning_rate)
  
def create_optimizer(trial):
    optimizer = trial.suggest_categorical('optimizer', ['sgd', 'adam'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    if optimizer[0] == 'sgd':
      return optim.SGD(model.parameters(), lr=learning_rate)
    else:
      return optim.Adam(model.parameters(), lr=learning_rate)

In [0]:
def load_data():
  digits = load_digits()
  data = digits['data']
  target = digits['target']
  trX, teX, trY, teY = train_test_split(data, target, test_size=0.2, random_state=0)
  trX = torch.from_numpy(trX).float()
  teX = torch.from_numpy(teX).float()
  trY = torch.from_numpy(trY.astype(np.int64))
  teY = torch.from_numpy(teY.astype(np.int64))
  return (trX, trY, teX, teY)

In [0]:
N_EPOCHS = 100
def objective(trial):
  trX, trY, teX, teY = load_data()
  model = LogisticNet(64, 10, trial)
  optimizer = create_optimizer(trial)
  loss_func = nn.CrossEntropyLoss()
  for epoch in range(N_EPOCHS):
      loss = train(model, loss_func, optimizer, trX, trY)
      val_loss, val_acc = valid(model, loss_func, teX, teY)
  return 1 - val_acc

In [0]:
optuna.logging.disable_default_handler()
study = optuna.create_study()
study.optimize(objective, n_trials=10)

In [120]:
print("best params: ", study.best_params)
print("best test accuracy: ", study.best_value)

best params:  {'hidden_dim': 128, 'dropout_rate': 0.07268943158505814, 'optimizer': 'sgd', 'learning_rate': 0.0008228873939678834}
best test accuracy:  0.875
